In [13]:
from deepfindET.utils import copick_tools
import copick
import os
import numpy as np
from patchify import patchify
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import keras._tf_keras.keras as keras

In [14]:
copick_config_path = "working/copick.json"
copick_root = copick.from_file(copick_config_path)

runs = os.listdir('Extracted_Data/train/static/ExperimentRuns')
voxel_size = 10
tomogram_algo = 'denoised'

out_name = 'remotetargets'
out_user_id = 'deepfindET'
out_session_id = '0'

In [15]:
def get_tomogram(run_id):
    return np.array(copick_tools.get_copick_tomogram(copick_root,
                                                voxelSize = voxel_size,
                                                tomoAlgorithm=tomogram_algo,
                                                tomoID=run_id))

def get_mask(run_id):
    return np.array(copick_tools.get_copick_segmentation(copick_root.get_run(run_id),
                                                segmentationName=out_name,
                                                userID=out_user_id,
                                                sessionID=out_session_id))

def shape_t(obj):
    obj = patchify(obj, (64, 64, 64), step=64)
    obj = np.reshape(obj, (-1, obj.shape[3], obj.shape[4], obj.shape[5]))
    return obj


def tomograms():
    tomogram = np.array([shape_t(get_tomogram(i)) for i in runs])
    tomogram = np.reshape(tomogram, (-1, tomogram.shape[2], tomogram.shape[3], tomogram.shape[4]))
    tomogram = np.stack((tomogram,)*3, axis=-1)
    return tomogram
def masks():
    mask = np.array([shape_t(get_mask(i)) for i in runs])
    mask = np.reshape(mask, (-1, mask.shape[2], mask.shape[3], mask.shape[4]))
    mask = to_categorical(masks,num_classes = 7)
    return mask
    

In [16]:
y_train,y_test = train_test_split(masks(), test_size = 0.2)

In [17]:
x_train, x_test = train_test_split(tomograms(), test_size=0.2)

In [18]:
# training
np.save('working/Temp_Data/train/x_train.npy', x_train)
np.save('working/Temp_Data/train/y_train.npy', y_train)

#testing
np.save('working/Temp_Data/test/x_test.npy', x_test)
np.save('working/Temp_Data/test/y_test.npy', y_test)

# Freeing the memory
del x_train, x_test, y_train, y_test

In [19]:
x_train, x_val, y_train, y_val = train_test_split(np.load('working/Temp_Data/train/x_train.npy', mmap_mode='r'),
                                                  np.load('working/Temp_Data/train/y_train.npy', mmap_mode='r'),
                                                  test_size = 0.2)
# training
np.save('working/Temp_Data/train/x_train.npy', x_train)
np.save('working/Temp_Data/train/y_train.npy', y_train)

#validation
np.save('working/Temp_Data/val/x_val.npy', x_val)
np.save('working/Temp_Data/val/y_val.npy', y_val)

del x_train, x_val, y_train, y_val

In [4]:
class generator(keras.utils.Sequence):
    def __init__(self, data, labels, batch_size):
        super(generator, self).__init__()
        self.x_train = data
        self.y_train = labels
        self.batch_size = batch_size

    def __len__(self):
        return (np.ceil(len(self.x_train) / self.batch_size)).astype(np.int16)

    def __getitem__(self, idx):
        batch_x = self.x_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [ ]:
x_train = np.load('working/Temp_Data/train/x_train.npy', mmap_mode='r')
y_train = np.load('working/Temp_Data/train/y_train.npy', mmap_mode='r')

x_val = np.load('working/Temp_Data/val/x_val.npy', mmap_mode='r')
y_val = np.load('working/Temp_Data/val/y_val.npy', mmap_mode='r')

train_generator = generator(x_train, y_train, 4)
val_generator = generator(x_val, y_val, 4)